# Reformat US Census data (+ other datasets!)




In [1]:
# import libraries
import numpy as np
import requests
import json
import pandas as pd
from pathlib import Path
import ast
from io import StringIO
import matplotlib.pyplot as plt
import geopandas as gpd
from sklearn.preprocessing import MinMaxScaler
from shapely.ops import nearest_points

import warnings
warnings.filterwarnings('ignore')

In [2]:
import reformat_census_vars as refor
import recs_EDA

In [3]:
path = '/global/scratch/users/cristina_crespo/p1_data/'

## 1) Import census data for all CTs

Num of census tract: https://www.census.gov/geographies/reference-files/time-series/geo/tallies.html

United States: 84,414	
Puerto Rico: 981	
Island Areas: 133	
Total: 85,528

CDD_TRACT has all of them - could check why we dont have 84k to start with !

In [4]:
census_df = pd.read_csv(path +'in_us_census/acs5_2020_vbs_per_ct.csv')

In [5]:
#make GEOID to create merges per CT
census_df['state'] = census_df['state'].astype(str).str.zfill(2)
census_df['county'] = census_df['county'].astype(str).str.zfill(3)
census_df['tract'] = census_df['tract'].astype(str).str.zfill(6)

census_df['GEOID'] = census_df[['state', 'county', 'tract']].astype(str).apply(lambda x: ''.join(x), axis=1)

#delete hawaii, alaska, puerto rico
census_df = census_df[~census_df.state.isin(['72'])]

In [6]:
len(census_df.state.unique())

51

We have CTS: 

In [7]:
len(census_df)

82907

## 2) Add other datasets to re-create all variables in RECS model

### 2.0 Subtract water bodies from Census tracts

In [8]:
#https://www.usgs.gov/national-hydrography/access-national-hydrography-products

I can onl get water bodies per state-county:
https://www.census.gov/cgi-bin/geo/shapefiles/index.php?year=2020&layergroup=Water
https://www2.census.gov/geo/tiger/TIGER2023/
--> Asked Chris Jones

### 2.1 Add CDD and HDD


In [9]:
#FEMA historical CDD nad HDD CT data
cdd_tract_raw = pd.read_csv(path +"in_other_datasets/CDD_HDD/CDDbytract.csv", encoding='utf-8',thousands=',').dropna()
hdd_tract_raw = pd.read_csv(path +"in_other_datasets/CDD_HDD/HDDbytract.csv", encoding='utf-8',thousands=',').dropna()

#rename cdd, hdd
cdd_tract = cdd_tract_raw[['GEOID','hist']]
cdd_tract.rename(columns = {'hist':'CDD'}, inplace = True)
hdd_tract = hdd_tract_raw[['GEOID','hist']]
hdd_tract.rename(columns = {'hist':'HDD'}, inplace = True)

#get geoid in strig format
cdd_tract['GEOID'] = cdd_tract['GEOID'].astype(str).str.zfill(11)
hdd_tract['GEOID'] = hdd_tract['GEOID'].astype(str).str.zfill(11)

In [10]:
#merge cdd, hedd with census data
enhanced_df  = census_df.merge(cdd_tract, on = 'GEOID', how = 'inner')
enhanced_df  = enhanced_df.merge(hdd_tract, on = 'GEOID', how = 'inner')

len(enhanced_df)

82907

### Merge with 2010 census tracts. 

Select the census tract in 2010 that has the biggest area overalp with the 2020 version

In [11]:
#read in crosswalk file for CTs
crosswalk_df = pd.read_csv('tab20_tract20_tract10_natl.txt', sep='|')

# Rename the GEOID column in enhanced_df to GEOID_20 for clarity
enhanced_df.rename(columns={'GEOID': 'GEOID_20'}, inplace=True)

# Select relevant columns from the crosswalk
crosswalk_df = crosswalk_df[['GEOID_TRACT_20', 'GEOID_TRACT_10', 'AREALAND_PART']]

# Make sure the columns are treated as strings
enhanced_df['GEOID_20'] = enhanced_df['GEOID_20'].astype(str)
crosswalk_df['GEOID_TRACT_20'] = crosswalk_df['GEOID_TRACT_20'].astype(str).str.zfill(11)
crosswalk_df['GEOID_TRACT_10'] = crosswalk_df['GEOID_TRACT_10'].astype(str).str.zfill(11)

# Sort by AREALAND_PART in descending order to get the tract with the largest overlap
crosswalk_df = crosswalk_df.sort_values(by='AREALAND_PART', ascending=False)
crosswalk_df = crosswalk_df.drop_duplicates(subset='GEOID_TRACT_20', keep='first')  # Keep largest area

# Merge enhanced_df with the crosswalk
enhanced_df = pd.merge(enhanced_df, crosswalk_df[['GEOID_TRACT_20', 'GEOID_TRACT_10']], how='left', left_on='GEOID_20', right_on='GEOID_TRACT_20')

# Drop the extra column and rename the 2010 GEOID column
enhanced_df.drop(columns=['GEOID_TRACT_20'], inplace=True)
enhanced_df.rename(columns={'GEOID_TRACT_10': 'GEOID_10'}, inplace=True)

# Verify the final result
print(f"Rows in original enhanced_df: {len(enhanced_df)}")
print(f"Rows in enhanced_df after crosswalk: {len(enhanced_df)}")


Rows in original enhanced_df: 82907
Rows in enhanced_df after crosswalk: 82907


In [12]:
# Check if all GEOIDs in enhanced_df exist in crosswalk_df
unmatched_geoids = enhanced_df[~enhanced_df['GEOID_20'].isin(crosswalk_df['GEOID_TRACT_20'])]
print(f"Number of unmatched 2020 tracts: {len(unmatched_geoids)}")
print('If none, correct crosswalk file used')

Number of unmatched 2020 tracts: 0
If none, correct crosswalk file used


### 2.2 Add energy costs

In [13]:
# Load and clean energy cost data
energy_costs_fuel_raw = pd.read_excel(path + "in_other_datasets/energy_costs/energycosts3.xlsx", thousands=',').dropna()

# Convert FIP to string format and pad with leading zeros to 11 characters
energy_costs_fuel_raw['FIP'] = energy_costs_fuel_raw['FIP'].astype(int).astype(str).str.zfill(11)

# Rename and select relevant columns
energy_costs_fuel_raw.rename(columns={'FIP': 'GEOID_10', 'ELEP': 'cost_EL', 'GASP': 'cost_NG', 'FULP': 'cost_OF'}, inplace=True)

# Inflate energy costs to 2020 dollars, save
energy_costs_fuel_raw[['cost_EL', 'cost_NG', 'cost_OF']] *= 1.0307
energy_costs_fuel = energy_costs_fuel_raw[['GEOID_10', 'cost_EL', 'cost_NG', 'cost_OF']]

# Merge energy cost data with enhanced_df using GEOID_2020
enhanced_df = enhanced_df.merge(energy_costs_fuel[['GEOID_10', 'cost_EL', 'cost_NG', 'cost_OF']], 
                                left_on='GEOID_10', right_on='GEOID_10', how='left')

# Check for rows with missing energy costs after merge
missing_rows = enhanced_df['cost_EL'].isna().sum()
print(f"Number of rows with missing energy costs after initial merge: {missing_rows}")

### 1. Fill missing values with COUNTY averages
if missing_rows > 0:
    # Extract county-level GEOID (first 5 digits of GEOID)
    enhanced_df['county_GEOID_20'] = enhanced_df['GEOID_20'].str[:5]

    # Calculate county-level averages for missing costs
    county_avg = enhanced_df.groupby('county_GEOID_20')[['cost_EL', 'cost_NG', 'cost_OF']].mean().reset_index()

    # Fill missing values by mapping county-level averages to the rows with missing data
    for col in ['cost_EL', 'cost_NG', 'cost_OF']:
        enhanced_df[col] = enhanced_df.apply(
            lambda row: county_avg.loc[county_avg['county_GEOID_20'] == row['county_GEOID_20'], col].values[0]
            if pd.isna(row[col]) and row['county_GEOID_20'] in county_avg['county_GEOID_20'].values
            else row[col], axis=1
        )

    
# Check for rows still missing after filling county averages
missing_rows_after_county = enhanced_df['cost_EL'].isna().sum()
print(f"Number of rows still missing energy costs after applying county averages: {missing_rows_after_county}")

### 2. Fill remaining missing values with STATE averages
if missing_rows_after_county > 0:
    # Extract state-level GEOID (first 2 digits of GEOID)
    enhanced_df['state_GEOID'] = enhanced_df['GEOID_20'].str[:2]

    # Calculate state-level averages for missing costs
    state_avg = enhanced_df.groupby('state_GEOID')[['cost_EL', 'cost_NG', 'cost_OF']].mean().reset_index()

    # Fill remaining missing values with state-level averages
    for col in ['cost_EL', 'cost_NG', 'cost_OF']:
        enhanced_df[col] = enhanced_df.apply(
            lambda row: state_avg.loc[state_avg['state_GEOID'] == row['state_GEOID'], col].values[0]
            if pd.isna(row[col]) and row['state_GEOID'] in state_avg['state_GEOID'].values
            else row[col], axis=1
        )

# Check for rows still missing after filling state averages
final_missing_rows = enhanced_df['cost_EL'].isna().sum()
print(f"Number of rows still missing energy costs after applying state averages: {final_missing_rows}")

# Final check and summary
if final_missing_rows > 0:
    print(f"Warning: {final_missing_rows} rows still missing energy costs after all filling attempts.")
else:
    print("All missing energy costs have been filled successfully!")

Number of rows with missing energy costs after initial merge: 49
Number of rows still missing energy costs after applying county averages: 15
Number of rows still missing energy costs after applying state averages: 0
All missing energy costs have been filled successfully!


In [14]:
# Check if all GEOIDs in enhanced_df exist in energy_costs_fuel
unmatched_geoids = enhanced_df[~enhanced_df['GEOID_10'].isin(energy_costs_fuel['GEOID_10'])]
print(f"Number of unmatched 2010 tracts: {len(unmatched_geoids)}")

Number of unmatched 2010 tracts: 49


In [15]:
len(enhanced_df)

82907

### 2.3 Add County/hh member AMI poverty levels


In [16]:
# Load AMI (Area Median Income) data per county using 2010 GEOIDs and drop missing values
ami_county_raw = pd.read_excel(path + "in_other_datasets/AMI_county_hh_members/FY2020NSP_IncomeLimits.xlsx").dropna()

# Filter out states not included in the analysis (e.g., Puerto Rico - 72)
excluded_states = ['66', '72', '78']
ami_county_raw = ami_county_raw[~ami_county_raw['State'].isin(excluded_states)]

# Create county-level GEOID using state and county codes
ami_county_raw['State'] = ami_county_raw['State'].astype(str).str.zfill(2)
ami_county_raw['County'] = ami_county_raw['County'].astype(str).str.zfill(3)
ami_county_raw['county_GEOID_10'] = ami_county_raw['State'] + ami_county_raw['County']

# Select relevant columns (GEOID_county and AMI limits for different household sizes)
ami_county = ami_county_raw[['county_GEOID_10', 'lim120_20p1', 'lim120_20p2', 'lim120_20p3',
                             'lim120_20p4', 'lim120_20p5', 'lim120_20p6', 'lim120_20p7', 
                             'lim120_20p8']]

# Aggregate to county-level by taking the mean where necessary
ami_county = ami_county.groupby('county_GEOID_10').mean().reset_index()

# Extract 2010 county GEOIDs from enhanced_df for merging
enhanced_df['county_GEOID_10'] = enhanced_df['GEOID_10'].str[:5]

# Merge enhanced_df with county-level AMI data
enhanced_df = enhanced_df.merge(ami_county, left_on='county_GEOID_10', right_on='county_GEOID_10', how='left')

# Check how many rows have missing AMI data after county-level merge
missing_rows_county = enhanced_df['lim120_20p1'].isna().sum()
print(f"Number of rows missing AMI data after county-level merge: {missing_rows_county}")

### 1. Fill missing values with STATE-level averages if county-level data is not available

if missing_rows_county > 0:
    
    # Calculate state-level averages for missing AMI values
    state_avg = ami_county_raw.copy()
    state_avg['state_GEOID'] = state_avg['county_GEOID_10'].str[:2]
    state_avg = state_avg.groupby('state_GEOID')[['lim120_20p1', 'lim120_20p2', 'lim120_20p3',
                                                  'lim120_20p4', 'lim120_20p5', 'lim120_20p6', 
                                                  'lim120_20p7', 'lim120_20p8']].mean().reset_index()

    # Fill missing rows using state-level averages
    for col in ['lim120_20p1', 'lim120_20p2', 'lim120_20p3', 'lim120_20p4', 
                'lim120_20p5', 'lim120_20p6', 'lim120_20p7', 'lim120_20p8']:
        enhanced_df[col] = enhanced_df.apply(
            lambda row: state_avg.loc[state_avg['state_GEOID'] == row['state_GEOID'], col].values[0]
            if pd.isna(row[col]) and row['state_GEOID'] in state_avg['state_GEOID'].values
            else row[col], axis=1
        )

    # Check how many rows have been filled using state-level averages
    missing_rows_state = enhanced_df['lim120_20p1'].isna().sum()
    rows_filled_by_state = missing_rows_county - missing_rows_state
    print(f"Number of rows filled using state-level averages: {rows_filled_by_state}")

# Double-check that the final row count matches the original row count
final_row_count = len(enhanced_df)
initial_row_count = len(enhanced_df['GEOID_20'])
if final_row_count != initial_row_count:
    raise ValueError(f"Row count mismatch: Expected {initial_row_count}, but got {final_row_count}")
else:
    print(f"Row count matches: {final_row_count} rows in the final dataframe.")

### 2. Select AMI limit based on household members
# Round average household members to the nearest integer
enhanced_df['hh_mem'] = enhanced_df['Average household members'].round().astype(int)

# Select appropriate AMI limit based on household size
enhanced_df['lim_income_120AMI'] = enhanced_df.apply(
    lambda row: row['lim120_20p' + str(min(row['hh_mem'], 8))], axis=1
)

# Drop unnecessary AMI columns after selecting the relevant one
enhanced_df.drop(columns=['lim120_20p1', 'lim120_20p2', 'lim120_20p3', 'lim120_20p4',
                          'lim120_20p5', 'lim120_20p6', 'lim120_20p7', 'lim120_20p8'], inplace=True)

### 3. Label homes as LMI (Low-to-Moderate Income) or not based on 120% AMI
enhanced_df['LMI'] = np.where(enhanced_df['Average household income'] <= enhanced_df['lim_income_120AMI'], 1, 0)

# Final check for row count consistency
assert len(enhanced_df) == initial_row_count, "Error: Row count mismatch after processing."
print(f" Final row count remains consistent: {len(enhanced_df)} rows.")

Number of rows missing AMI data after county-level merge: 6
Number of rows filled using state-level averages: 6
Row count matches: 82907 rows in the final dataframe.
 Final row count remains consistent: 82907 rows.


In [17]:
len(enhanced_df)

82907

### 2.4 Add Census Divisions 

In RECS, this is the labelling 

-1 New England
-2 Middle Atlantic
-3 East North Central
-4 West North Central
-5 South Atlantic
-6 East South Central
-7 West South Central
-8 Mountain North
-9 Mountain South
-10 Pacific

https://www2.census.gov/geo/pdfs/maps-data/maps/reference/us_regdiv.pdf 

Note: there are 2 Mountain regions (N/S) - "Mountain North" (CO, ID, MT, UT, and WY) and "Mountain South" (AZ, NM, and NV) https://www.eia.gov/consumption/residential/reports/2009/16-states.php

In [18]:
#no puerto rico here

state_to_census_region = {'01':6, '02':10, '04':9, '05':7, '06':10, '08':8, '09':1, '10':5,'11':5, '12':5,
             '13':5, '15':10, '16':8, '17':3, '18':3, '19':4, '20':4, '21':6, '22':7,
            '23':1, '24':5, '25':1, '26':3, '27':4, '28':6, '29':4, '30':8, '31':4,
            '32':9, '33':1, '34':2, '35':9, '36':2, '37':5, '38':4, '39':3, '40':7,
            '41':10, '42':2, '44':1, '45':5, '46':4, '47':6, '48':7, '49':8, '50':1,
            '51':5, '53':10, '54':5, '55':3, '56':8}

reg_to_region_dict = {
    1: 'New England',
    2: 'Middle Atlantic',
    3: 'East North Central',
    4: 'West North Central',
    5: 'South Atlantic',
    6: 'East South Central',
    7: 'West South Central',
    8: 'Mountain North',
    9: 'Mountain South',
    10: 'Pacific'
}
enhanced_df['DIVISION'] =  enhanced_df["state"].map(state_to_census_region)
enhanced_df['DIVISION'] =  enhanced_df["DIVISION"].map(reg_to_region_dict)

In [19]:
len(enhanced_df)

82907

### 2.5 Add rural/urban


In [20]:
# Load the rural-urban population data from CSV
urban_rural_df_raw = pd.read_csv(path + "in_other_datasets/rural_urban/PctUrbanRural_County.csv", sep=";")

# Select relevant columns for analysis
urban_rural_df = urban_rural_df_raw[['STATE', 'COUNTY', 'COUNTYNAME', 'STATENAME',
                                     'POPPCT_RURAL', 'POPPCT_UC', 'POPPCT_UA',
                                     'POP_RURAL', 'POP_UC', 'POP_UA']]

# Create county-level GEOID by combining STATE and COUNTY codes
urban_rural_df['STATE'] = urban_rural_df['STATE'].astype(str).str.zfill(2)
urban_rural_df['COUNTY'] = urban_rural_df['COUNTY'].astype(str).str.zfill(3)
urban_rural_df['county_GEOID_10'] = urban_rural_df['STATE'] + urban_rural_df['COUNTY']

# Convert percentage columns to fractions (divide by 100)
urban_rural_df[['frac_RURAL', 'frac_UC', 'frac_UA']] = urban_rural_df[['POPPCT_RURAL', 'POPPCT_UC', 'POPPCT_UA']].div(100)

# Merge rural-urban fractions with enhanced_df
enhanced_df = enhanced_df.merge(
    urban_rural_df[['county_GEOID_10', 'frac_RURAL', 'frac_UC', 'frac_UA']],
    how='left',  # Use 'left' merge to retain all rows in enhanced_df
    left_on='county_GEOID_10',
    right_on='county_GEOID_10'
)

# Check for missing values after merge
missing_rural_data = enhanced_df['frac_RURAL'].isna().sum()
print(f"Number of rows missing rural-urban data after merge: {missing_rural_data}")

# If there are missing values, fill them with state-level averages
if missing_rural_data > 0:
    # Create state-level GEOID for enhanced_df and urban_rural_df
    urban_rural_df['state_GEOID'] = urban_rural_df['county_GEOID_10'].str[:2]

    # Calculate state-level averages for frac_RURAL, frac_UC, and frac_UA
    state_avg = urban_rural_df.groupby('state_GEOID')[['frac_RURAL', 'frac_UC', 'frac_UA']].mean().reset_index()

    # Fill missing values using state-level averages
    for col in ['frac_RURAL', 'frac_UC', 'frac_UA']:
        enhanced_df[col] = enhanced_df.apply(
            lambda row: state_avg.loc[state_avg['state_GEOID'] == row['state_GEOID'], col].values[0]
            if pd.isna(row[col]) and row['state_GEOID'] in state_avg['state_GEOID'].values
            else row[col], axis=1
        )

    # Count the number of rows filled using state-level averages
    remaining_missing_rows = enhanced_df['frac_RURAL'].isna().sum()
    rows_filled_by_state = missing_rural_data - remaining_missing_rows
    print(f"Number of rows filled using state-level averages: {rows_filled_by_state}")


# Final consistency check
if len(enhanced_df) != len(enhanced_df['GEOID_20']):
    raise ValueError(f"Row count mismatch: Expected {len(enhanced_df['GEOID_20'])}, but got {len(enhanced_df)}.")
else:
    print(f"Row count is consistent after filling missing data: {len(enhanced_df)} rows.")

Number of rows missing rural-urban data after merge: 0
Row count is consistent after filling missing data: 82907 rows.


In [21]:
len(enhanced_df)

82907

### 2.6 Add energy prices per state


In [22]:
#Add new 2020 energy prices form form 861
#prices per utility
e_prices = pd.read_excel(path+'in_other_datasets/energy_prices/Sales_Ult_Cust_2020.xlsx', skiprows = 2)
e_prices.drop(e_prices.tail(1).index,inplace=True)


In [23]:
#Take away canadian pricesss from ME- Maine
#https://www.eia.gov/electricity/gridmonitor/about
condition = (e_prices['Utility Number'] == 1179) & (e_prices['BA Code'] =='NBSO')
e_prices = e_prices[~condition] 

In [24]:
#only bundled service type: delivery + energy
e_prices = e_prices[e_prices['Service Type'] == 'Bundled']

e_prices_final = e_prices[['Utility Number', 'Utility Name', 'Service Type', 
                           'State', 'Thousand Dollars', 'Megawatthours', 'Count']]

#re-insert nans 
e_prices_final = e_prices_final.replace('nan', np.nan)
e_prices_final = e_prices_final.replace('', np.nan)
e_prices_final = e_prices_final.replace('.', np.nan)
e_prices_final = e_prices_final.replace(0, np.nan)
    
#Drop all nans in vb of interest              
e_prices_final = e_prices_final.dropna(subset=['State', 'Thousand Dollars', 'Megawatthours' ], how='any')
print(len(e_prices_final)/len(e_prices))

0.9224932249322493


In [25]:
#totals per state
res_prices_state = e_prices_final.groupby('State').sum()

#gen prices per state
res_prices_state['elec_price_kwh'] = res_prices_state['Thousand Dollars'].div(res_prices_state['Megawatthours'])
res_prices_state.reset_index(inplace = True)

In [26]:
state_fips_dict = {
    'AL': '01', 'AK': '02', 'AZ': '04', 'AR': '05', 'CA': '06',
    'CO': '08', 'CT': '09', 'DE': '10', 'DC': '11', 'FL': '12', 'GA': '13',
    'HI': '15', 'ID': '16', 'IL': '17', 'IN': '18', 'IA': '19',
    'KS': '20', 'KY': '21', 'LA': '22', 'ME': '23', 'MD': '24',
    'MA': '25', 'MI': '26', 'MN': '27', 'MS': '28', 'MO': '29',
    'MT': '30', 'NE': '31', 'NV': '32', 'NH': '33', 'NJ': '34',
    'NM': '35', 'NY': '36', 'NC': '37', 'ND': '38', 'OH': '39',
    'OK': '40', 'OR': '41', 'PA': '42', 'RI': '44', 'SC': '45',
    'SD': '46', 'TN': '47', 'TX': '48', 'UT': '49', 'VT': '50',
    'VA': '51', 'WA': '53', 'WV': '54', 'WI': '55', 'WY': '56'
}

# Map FIPS codes to a new column 'state'
res_prices_state['state']  = res_prices_state['State'].map(state_fips_dict)
len(res_prices_state)

51

In [27]:
#merge prics
enhanced_df = enhanced_df.merge(res_prices_state[['state','elec_price_kwh' ]], left_on = 'state' , right_on= 'state', how= 'inner')

print(len(enhanced_df))

82907


In [28]:
# Load energy prices and drop NaN values
of_energy_prices = pd.read_excel(path + "in_other_datasets/energy_prices/table_joule_e_prices.xlsx", skiprows=1).dropna()

# Create a DataFrame with new rows for Alaska (AK) and Hawaii (HI)
new_rows = pd.DataFrame([
    {'State': 'HI', 'NG ($/THERM)': 0, 'ELEC ($/KWH)': 0,
     'FO ($/GAL)': of_energy_prices['FO ($/GAL)'].median(),
     'PROPANE ($/GAL)': of_energy_prices['PROPANE ($/GAL)'].median()},
    
    {'State': 'AK', 'NG ($/THERM)': 0, 'ELEC ($/KWH)': 0,
     'FO ($/GAL)': of_energy_prices['FO ($/GAL)'].median(),
     'PROPANE ($/GAL)': of_energy_prices['PROPANE ($/GAL)'].median()}
])

# Use pd.concat to combine the original DataFrame and the new rows
of_energy_prices = pd.concat([of_energy_prices, new_rows], ignore_index=True)

# Map FIPS codes to a new column 'state' using the state_fips_dict
of_energy_prices['state'] = of_energy_prices['State'].map(state_fips_dict)

# Merge with enhanced_df to add fuel oil and propane costs
enhanced_df = enhanced_df.merge(
    of_energy_prices[['state', 'FO ($/GAL)', 'PROPANE ($/GAL)']], 
    on='state', 
    how='inner'
)

In [29]:
len(enhanced_df)

82907

In [30]:
prices_gas = pd.read_excel(path +'in_other_datasets/energy_prices/gas_prices_state_eia.xlsx', skiprows = 2)

# Dictionary mapping US state names to state abbreviations
state_abbr = {
    'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ', 'Arkansas': 'AR', 'California': 'CA',
    'Colorado': 'CO', 'Connecticut': 'CT', 'Delaware': 'DE', 'Florida': 'FL', 'Georgia': 'GA',
    'Hawaii': 'HI', 'Idaho': 'ID', 'Illinois': 'IL', 'Indiana': 'IN', 'Iowa': 'IA',
    'Kansas': 'KS', 'Kentucky': 'KY', 'Louisiana': 'LA', 'Maine': 'ME', 'Maryland': 'MD',
    'Massachusetts': 'MA', 'Michigan': 'MI', 'Minnesota': 'MN', 'Mississippi': 'MS',
    'Missouri': 'MO', 'Montana': 'MT', 'Nebraska': 'NE', 'Nevada': 'NV', 'New Hampshire': 'NH',
    'New Jersey': 'NJ', 'New Mexico': 'NM', 'New York': 'NY', 'North Carolina': 'NC',
    'North Dakota': 'ND', 'Ohio': 'OH', 'Oklahoma': 'OK', 'Oregon': 'OR', 'Pennsylvania': 'PA',
    'Rhode Island': 'RI', 'South Carolina': 'SC', 'South Dakota': 'SD', 'Tennessee': 'TN',
    'Texas': 'TX', 'Utah': 'UT', 'Vermont': 'VT', 'Virginia': 'VA', 'Washington': 'WA',
    'West Virginia': 'WV', 'Wisconsin': 'WI', 'Wyoming': 'WY', 'District of Columbia' : 'DC'
}

# Add a new column 'state_abbr' with state abbreviations
prices_gas['state_abbr'] = prices_gas['Location'].map(state_abbr)

In [31]:
prices_gas_state = prices_gas[[2020,'state_abbr' ]]
prices_gas_state.rename(columns = {2020:'gas_price_th_ft3'}, inplace = True)
prices_gas_state = prices_gas_state[prices_gas_state.state_abbr.notna()]

In [32]:
#merge gas prices per state
# Map FIPS codes to a new column 'state'
prices_gas_state['state_fips']  = prices_gas_state['state_abbr'].map(state_fips_dict)
enhanced_df = enhanced_df.merge(prices_gas_state[['state_fips', 'gas_price_th_ft3']],  left_on = 'state' , right_on= 'state_fips', how= 'inner')

len(enhanced_df)

82907

In [33]:
#save other fuels
enhanced_df['Fuel_other'] = enhanced_df['Propane (bottled gas)']+enhanced_df['Other_fuel'] +enhanced_df['Not applicable']

In [34]:
# Drop multiple columns from enhanced_df
enhanced_df.drop(columns=['Unnamed: 0', 'state_GEOID'], inplace=True)

### 2.7 Add climate zones


In [35]:
# --- File Paths ---
climate_zones_path = path + "in_other_datasets/climate_zones/Climate_Zones_-_DOE_Building_America_Program.shp"
gis_ct_path = path + "in_us_census/2020_gis_tracts/2020_us_tracts.shp"

# --- Load Climate Zones and Census Tract GIS Data ---
# Read spatial files with EPSG:4326 CRS
climate_zones = gpd.read_file(climate_zones_path).to_crs(epsg=4326)
gis_ct = gpd.read_file(gis_ct_path).to_crs(epsg=4326)

# --- Overlay to Find Intersections between Census Tracts and Climate Zones ---
df_intersect = gpd.overlay(gis_ct, climate_zones, how='intersection')

# Calculate area of overlap and keep the largest area for each tract
df_intersect['area'] = df_intersect.geometry.area
df_intersect = df_intersect.sort_values(by='area', ascending=False).drop_duplicates(subset='GEOID', keep='first')
df_intersect.drop(columns=['area'], inplace=True)

# --- Create IECC Climate Code Label ---
df_intersect['IECC_climate_code'] = df_intersect['IECC_Clima'].astype(str) + df_intersect['IECC_Moist']

# --- Add HAWAII Climate Zone 1A ---
enhanced_df_HI = enhanced_df[enhanced_df['state'] == '15'].copy()
enhanced_df_HI['IECC_climate_code'] = '1A'

# Drop original Hawaii rows from enhanced_df
enhanced_df_no_HI = enhanced_df[enhanced_df['state'] != '15']

# Combine back with updated Hawaii rows
enhanced_df = pd.concat([enhanced_df_no_HI, enhanced_df_HI], ignore_index=True)

# --- Add ALASKA Climate Zones (Zone 7 and Zone 8) ---
# Mapping from location to county and FIPS
location_to_county = {
    'Aleutians East Borough': 'Aleutians East',
    'Aleutians West Census Area': 'Aleutians West',
    'Southeast Fairbanks Census Area': 'Southeast Fairbanks',
    'Dillingham Census Area': 'Dillingham',
    'North Slope Borough': 'North Slope',
    'Yakutat plus Hoonah-Angoon': 'Hoonah-Angoon',
    'Kodiak Island Borough': 'Kodiak Island',
    'Skagway Municipality': 'Skagway-Hoonah-Angoon',
    'Northwest Arctic Borough': 'Northwest Arctic',
    'Ketchikan Gateway Borough': 'Ketchikan Gateway',
    'Kusilvak Census Area': 'Kusilvak',
    'Fairbanks North Star Borough': 'Fairbanks North Star',
    'Juneau City and Borough': 'Juneau',
    'Nome Census Area': 'Nome',
    'Prince of Wales-Hyder Census Area': 'Prince of Wales-Hyder',
    'Sitka City and Borough': 'Sitka',
    'Petersburg Borough': 'Petersburg',
    'Wrangell City and Borough': 'Wrangell',
    'Matanuska-Susitna Borough': 'Matanuska-Susitna',
    'Yukon-Koyukuk Census Area': 'Yukon-Koyukuk',
    'Kenai Peninsula Borough': 'Kenai Peninsula',
    'Haines Borough': 'Haines',
    'Bristol Bay plus Lake and Peninsula': 'Lake and Peninsula',
    'Bethel Census Area': 'Bethel',
    'Denali Borough': 'Denali',
    'Anchorage Municipality': 'Anchorage',
    'Valdez-Cordova Census Area': 'Valdez-Cordova'
}

county_to_fips = {
    'Aleutians East': '013',
    'Aleutians West': '016',
    'Southeast Fairbanks': '240',
    'Dillingham': '070',
    'North Slope': '185',
    'Hoonah-Angoon': '105',
    'Kodiak Island': '150',
    'Skagway-Hoonah-Angoon': '230',
    'Northwest Arctic': '188',
    'Ketchikan Gateway': '130',
    'Kusilvak': '158',
    'Fairbanks North Star': '090',
    'Juneau': '110',
    'Nome': '180',
    'Prince of Wales-Hyder': '201',
    'Sitka': '220',
    'Petersburg': '195',
    'Wrangell': '275',
    'Matanuska-Susitna': '170',
    'Yukon-Koyukuk': '290',
    'Kenai Peninsula': '122',
    'Haines': '100',
    'Lake and Peninsula': '164',
    'Bethel': '050',
    'Denali': '068',
    'Anchorage': '020',
    'Valdez-Cordova': '261'
}

# Boroughs in Zone 8 (special cases)
boroughs_zone_8 = [
    'Bethel Census Area', 'Northwest Arctic Borough', 'Dillingham Census Area',
    'Southeast Fairbanks Census Area', 'Fairbanks North Star Borough',
    'North Slope Borough', 'Kusilvak Census Area', 'Nome Census Area',
    'Yukon-Koyukuk Census Area'
]

# Get FIPS codes for Zone 8 Boroughs
fips_8AK = [county_to_fips[location_to_county[borough]] for borough in boroughs_zone_8]

# Assign climate zones to Alaska rows based on county FIPS
enhanced_df.loc[(enhanced_df['county'].isin(fips_8AK)) & (enhanced_df['state'] == '02'), 'IECC_climate_code'] = '8AK'
enhanced_df.loc[(~enhanced_df['county'].isin(fips_8AK)) & (enhanced_df['state'] == '02'), 'IECC_climate_code'] = '7AK'

# --- Initial Merge with Climate Zones ---
print(f"GEOIDs in enhanced_df before merge: {enhanced_df['GEOID_20'].nunique()}")
print(f"GEOIDs in df_intersect: {df_intersect['GEOID'].nunique()}")

# Merge and avoid duplicates using suffixes
enhanced_df = enhanced_df.merge(df_intersect[['GEOID', 'IECC_climate_code']],
                                 left_on='GEOID_20', right_on='GEOID', 
                                 how='left', suffixes=('', '_new'))

# --- Fix Duplicate Columns ---
# If IECC_climate_code already exists, fill NaN values with new data
if 'IECC_climate_code_new' in enhanced_df.columns:
    enhanced_df['IECC_climate_code'] = enhanced_df['IECC_climate_code'].combine_first(enhanced_df['IECC_climate_code_new'])
    # Drop the extra column after merging
    enhanced_df.drop(columns=['IECC_climate_code_new', 'GEOID'], inplace=True)

# --- Check for Missing GEOIDs Only After Initial Merge ---
missing_geoids = enhanced_df[enhanced_df['IECC_climate_code'].isnull()]
print(f"Number of missing GEOIDs after initial merge: {len(missing_geoids)}")

# --- Handle Missing GEOIDs Using Nearest Neighbor ---
if len(missing_geoids) > 0:
    print(f"Finding nearest climate zones for {len(missing_geoids)} unmatched tracts...")

    # Get unmatched census tracts from GIS file
    unmatched_geoids = gis_ct[gis_ct['GEOID'].isin(missing_geoids['GEOID_20'])].copy()

    # Precompute nearest climate zone for unmatched tracts
    nearest_climate_codes = []
    for index, row in unmatched_geoids.iterrows():
        point = row.geometry
        nearest_zone_index = climate_zones.distance(point).idxmin()
        nearest_zone = climate_zones.iloc[nearest_zone_index]

        # Create the climate code for the nearest zone
        nearest_climate_codes.append({
            'GEOID_20': row['GEOID'],
            'IECC_climate_code': str(nearest_zone['IECC_Clima']) + str(nearest_zone['IECC_Moist'])
        })

    # Create DataFrame with nearest climate matches
    nearest_climate_df = pd.DataFrame(nearest_climate_codes)

    # Merge nearest climate zones with unmatched rows
    enhanced_df = enhanced_df.merge(nearest_climate_df, on='GEOID_20', how='left', suffixes=('', '_nearest'))

    # Fill missing values from nearest match
    enhanced_df['IECC_climate_code'] = enhanced_df['IECC_climate_code'].combine_first(enhanced_df['IECC_climate_code_nearest'])

    # Drop unnecessary columns
    enhanced_df.drop(columns=['IECC_climate_code_nearest'], inplace=True)

# --- Final Sanity Checks ---
# Ensure that all original rows are retained and no duplicates are introduced
print(f"Rows in enhanced_df after filling missing climates: {len(enhanced_df)}")
print(f"Unique GEOIDs after merge: {enhanced_df['GEOID_20'].nunique()} (Should match original count)")

# Fill any remaining NaN climate codes with a placeholder if necessary
enhanced_df['IECC_climate_code'].fillna('Unknown', inplace=True)

GEOIDs in enhanced_df before merge: 82907
GEOIDs in df_intersect: 82683
Number of missing GEOIDs after initial merge: 1140
Finding nearest climate zones for 1140 unmatched tracts...
Rows in enhanced_df after filling missing climates: 82907
Unique GEOIDs after merge: 82907 (Should match original count)


In [36]:
len(enhanced_df)

82907

In [37]:
print('Fraction of CTs saved from contiguous US, 2020:', len(enhanced_df)/len(census_df))

Fraction of CTs saved from contiguous US, 2020: 1.0


In [38]:
print('Fraction of CTs saved from all US, 2020:', len(enhanced_df)/len(gis_ct)) 

Fraction of CTs saved from all US, 2020: 0.9693550650079507


## 3) Save all raw variables per census tract

In [39]:
#save all vaiables for later analysis 
enhanced_df.to_csv(path+'out_final/ct_enhanced_analysis_final.csv')

## 4) Match RECS X variable structure

In [40]:
#select only variables needed to apply models to
df_vars = enhanced_df[[ 'county', 'tract', 'CDD', 'HDD', 'Average year built','Average num bedrooms', 'Average num rooms',
                           'Average household members', 'Average household income', 'cost_EL', 'cost_NG','cost_OF',
                           '5 or more units', 'Mobile home', 'Attached, 1 unit', 'Detached, 1 unit','IECC_climate_code',
                           'Renter occupied','Electricity', 'Fuel oil', 'Natural gas from underground pipes', 'Fuel_other',
                           'Wood or pellets', 'DIVISION', 'state', 'frac_RURAL', 'frac_UA', 'frac_UC', 'total_population']]
                           

In [41]:
#make dummies if needed
df_vars = pd.concat([df_vars, pd.get_dummies(pd.Series(df_vars.state), drop_first=False, prefix='STATE_FIPS')] , axis=1)
df_vars = pd.concat([df_vars, pd.get_dummies(pd.Series(df_vars.IECC_climate_code), drop_first=False, prefix='IECC_climate_code')] , axis=1)
df_vars = pd.concat([df_vars, pd.get_dummies(pd.Series(df_vars.DIVISION), drop_first=False, prefix='Census Div')] , axis=1)


In [42]:
#there are some climate zones and states represented in the RECS that we dont havein th acs 
df_vars['IECC_climate_code_7A'] = np.zeros(len(df_vars))
df_vars['IECC_climate_code_7B']= np.zeros(len(df_vars))

#put in value for CO and WY that had zone z of unknown humidiy level
#7A - 27, 38, 55, 23, 26 
#7B - 8, 56
df_vars.loc[(df_vars['IECC_climate_code_7N/A'] == 1) & (df_vars['state'].isin(['27', '38', '55', '23', '26' ])), 'IECC_climate_code_7A'] = 1
df_vars.loc[(df_vars['IECC_climate_code_7N/A'] == 1) & (df_vars['state'].isin(['08', '56' ])), 'IECC_climate_code_7B'] = 1


In [43]:
#dummies that have been dropped
#'2 to 4 units',
# 'Owner occupied',
# East North Central 
#'frac_UC',
#'IECC_climate_code_1A',
#'STATE_FIPS_01',
#drop the IECC_climate_code_7N/A and IECC_climate_code_8N/A categories

df_vars_clean = df_vars[['state','county', 'tract', 'CDD', 'HDD', 'Average year built',
       'Average num bedrooms', 'Average num rooms',
       'Average household members', 'Average household income', 'cost_EL',
       'cost_NG', 'cost_OF', '5 or more units', 'Mobile home',
       'Attached, 1 unit', 'Detached, 1 unit', 'IECC_climate_code_2A','IECC_climate_code_2B', 
       'IECC_climate_code_3A','IECC_climate_code_3B', 'IECC_climate_code_3C', 'IECC_climate_code_4A',
       'IECC_climate_code_4B', 'IECC_climate_code_4C', 'IECC_climate_code_5A',
       'IECC_climate_code_5B', 'IECC_climate_code_6A', 'IECC_climate_code_6B',
       'IECC_climate_code_7A', 'IECC_climate_code_7AK', 'IECC_climate_code_7B', 'IECC_climate_code_8AK',
       'Renter occupied', 'Electricity', 'Fuel oil',
       'Natural gas from underground pipes', 'Fuel_other', 'Wood or pellets',
       'Census Div_East South Central', 'Census Div_Middle Atlantic',
       'Census Div_Mountain North', 'Census Div_Mountain South',
       'Census Div_New England', 'Census Div_Pacific',
       'Census Div_South Atlantic', 'Census Div_West North Central',
       'Census Div_West South Central','STATE_FIPS_02','STATE_FIPS_04',
       'STATE_FIPS_05','STATE_FIPS_06','STATE_FIPS_08','STATE_FIPS_09',
       'STATE_FIPS_10','STATE_FIPS_11','STATE_FIPS_12','STATE_FIPS_13',
       'STATE_FIPS_15', 'STATE_FIPS_16','STATE_FIPS_17','STATE_FIPS_18',
       'STATE_FIPS_19','STATE_FIPS_20','STATE_FIPS_21','STATE_FIPS_22',
       'STATE_FIPS_23','STATE_FIPS_24','STATE_FIPS_25','STATE_FIPS_26',
       'STATE_FIPS_27','STATE_FIPS_28','STATE_FIPS_29','STATE_FIPS_30',
       'STATE_FIPS_31','STATE_FIPS_32','STATE_FIPS_33','STATE_FIPS_34',
       'STATE_FIPS_35', 'STATE_FIPS_36','STATE_FIPS_37','STATE_FIPS_38',
       'STATE_FIPS_39','STATE_FIPS_40','STATE_FIPS_41','STATE_FIPS_42',
       'STATE_FIPS_44','STATE_FIPS_45','STATE_FIPS_46','STATE_FIPS_47',
       'STATE_FIPS_48','STATE_FIPS_49','STATE_FIPS_50','STATE_FIPS_51',
       'STATE_FIPS_53','STATE_FIPS_54','STATE_FIPS_55','STATE_FIPS_56','frac_RURAL', 'frac_UC',
        'total_population']]

### Reclassify numeric variables

- Average year built
- Average num bedrooms
- Average num rooms
- Average household income
- Average household members

In [44]:
#Year built
# Define the bin edges and labels
bins = [1945, 1950, 1960, 1970, 1980, 1990, 2000, 2010, float('inf')]
labels = [1945, 1955, 1965, 1975, 1985, 1995, 2005, 2015]

# Reclassify the values in the 'Year' column based on the conditions
df_vars_clean['Average year built'] = pd.cut(df_vars_clean['Average year built'], bins=bins, labels=labels)


In [45]:
#Bedrooms
# Define the bin edges and labels
bins = [0, 0.5, 1.5, 2.5, 3.5, 4.5, float('inf')]
labels = [0, 1, 2, 3, 4, 5]

# Reclassify the values in the 'Year' column based on the conditions
df_vars_clean['Average num bedrooms'] = pd.cut(df_vars_clean['Average num bedrooms'], bins=bins, labels=labels)

In [46]:
#Rooms
# Define the bin edges and labels
bins = [0, 0.5, 1.5, 2.5, 3.5, 4.5, 5.5, 6.5, 7.5, 8.5, float('inf')]
labels = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

# Reclassify the values in the 'Year' column based on the conditions
df_vars_clean['Average num rooms'] = pd.cut(df_vars_clean['Average num rooms'], bins=bins, labels=labels)


In [47]:
#HH members
# Define the bin edges and labels
bins = [0.5, 1.5, 2.5, 3.5, 4.5, 5.5, 6.5, float('inf')]
labels = [1, 2, 3, 4, 5, 6, 7]

# Reclassify the values in the 'Year' column based on the conditions
df_vars_clean['Average household members'] = pd.cut(df_vars_clean['Average household members'], bins=bins, labels=labels)


In [48]:
#Income
# Define the bin edges and labels
bins = [0, 10000, 15000, 20000, 25000, 30000, 35000, 40000, 50000, 60000, 75000, 100000, 150000, float('inf')]
labels = [5000, 12500, 17500, 22500, 27500, 32500, 37500, 45500, 55500, 67500, 87500, 125000, 175000]

# Reclassify the values in the 'Year' column based on the conditions
df_vars_clean['Average household income'] = pd.cut(df_vars_clean['Average household income'], bins=bins, labels=labels)


In [49]:
# Convert the data type of the columns from 'category' to 'float'
df_vars_clean[['Average year built',
       'Average num bedrooms', 'Average num rooms', 'Average household income',
       'Average household members']] = df_vars_clean[['Average year built',
       'Average num bedrooms', 'Average num rooms', 'Average household income',
       'Average household members']].astype(float)


# 4) Normalize ACS variables using RECS mean, std

In [50]:
#scaling VBS - get the average and std of the RECS vbs
RECS_raw = pd.read_csv(path + 'RECS/recs2020_public_v5.csv')

#get normalization parameters
X_sub, Y, RECS_norm_param = recs_EDA.vb_transform(RECS_raw) 

In [51]:
# Define a dictionary for renaming index values
index_rename_dict = {'HDD30YR_PUB': 'HDD', 'CDD30YR_PUB':'CDD', 'DOLLAREL':'cost_EL', 'DOLLARNG':'cost_NG', 'DOLLAROF':'cost_OF','YEARMADERANGE':'Average year built', 
                      'BEDROOMS':'Average num bedrooms', 'TOTROOMS':'Average num rooms', 'MONEYPY':'Average household income', 'NHSLDMEM':'Average household members'}

# Rename index values using the dictionary
RECS_norm_param = RECS_norm_param.rename(index=index_rename_dict)


In [52]:
#Test code to normalize recs and ACS data
cols_to_norm = ['HDD', 'CDD', 'cost_EL', 'cost_NG', 'cost_OF', 'Average year built',
       'Average num bedrooms', 'Average num rooms', 'Average household income',
       'Average household members']

# Normalize the selected columns in the original DataFrame (df)
df_vars_clean[cols_to_norm] = (df_vars_clean[cols_to_norm] - RECS_norm_param['Mean']) / RECS_norm_param['std']


## 5) Save outputs for model application

In [53]:
#save dataframe 
df_vars_clean.to_csv(path+'out_final/ct_enhanced_model_application_final.csv')